In [1]:
import IPython.display
import sys
from rdflib import *
from rdflib.collection import Collection
import glob
import os
import re
import random
import csv
import itertools
from SPARQLWrapper import SPARQLWrapper, JSON, BASIC

In [2]:
def getFirstAndFinalStates(auth, activity, scene):
    result = None
    queryString = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX : <http://example.org/virtualhome2kg/ontology/>
PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?startState ?startStateType ?startObject ?endState ?endStateType ?endObject WHERE {
    <%activity> :hasEvent ?startEvent, ?endEvent .
    ?startEvent a :StartEvent .
    ?endEvent a :EndEvent .
    ?startEvent :situationBeforeEvent ?startSituation ;
        ho:object ?startObject .
    ?startState :partOf ?startSituation ;
        :isStateOf ?startObject ;
        :state ?startStateType .
    ?endEvent :situationAfterEvent ?endSituation ;
        ho:object ?endObject .
    ?endState :partOf ?endSituation ;
        :isStateOf ?endObject ;
        :state ?endStateType .
}
    """
    sparql = SPARQLWrapper(auth[0] + "_event_" + scene)
    sparql.setHTTPAuth(BASIC)
#     sparql.setCredentials(auth[1], auth[2])
    queryString = queryString.replace("%activity", activity)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)

    try :
        json = sparql.query().convert()
        bindings = json['results']['bindings']
        result = bindings
    except  Exception as e:
        print(e)
    return result

In [3]:
def readRoutine(path):
    with open(path) as f:
        reader = csv.reader(f)
        list = [row for row in reader]
        return list

In [4]:
def getActivity(auth, activity_name, scene):
    result = []
    queryString = """
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        select distinct *
        where { 
            ?s a ho:%s ;
                rdfs:comment ?comment .
        } ORDER BY RAND() limit 1 
    """
    sparql = SPARQLWrapper(auth[0] + "_event_" + scene)
    sparql.setHTTPAuth(BASIC)
#     sparql.setCredentials(auth[1], auth[2])
    sparql.setQuery(queryString % activity_name)
    sparql.setReturnFormat(JSON)

    try :
        json = sparql.query().convert()
        bindings = json['results']['bindings']
        result.append(bindings[0]['s']['value'])
        result.append(bindings[0]['comment']['value'])
    except  Exception as e:
        print(e)
        result = e
    return result

In [5]:
def checkConsistency(activity1_value, activity2_value):
    finalStateList1 = activity1_value["endState"]
    firstStateList2 = activity2_value["startState"]
    executable = True
    for finalState in finalStateList1:
        finalStateObject = stateMap[finalState]["object"]
        finalStateTypes = stateMap[finalState]["state"]
        for firstState in firstStateList2:
            firstStateObject = stateMap[firstState]["object"]
            firstStateTypes = stateMap[firstState]["state"]
            if finalStateObject == firstStateObject:
                for finalStateType in finalStateTypes:
                    if finalStateType in inverse_states:
                        if inverse_states[finalStateType] in firstStateTypes:
                            #print([finalStateType, firstStateTypes])
                            #さらに、そのobjectがactivtyの対象になっているかチェックし、対象となっていなければ上書きする
                            executable = checkTargetObject(firstState)
                            assert executable
                    else:
                        #print([finalState, firstState])
                        continue
    return executable

In [6]:
def checkTargetObject(auth, state):
    queryString = """
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        ASK { 
            <%s> :partOf ?situation ;
                      :isStateOf ?object .
            ?action (:situationBeforeAction|:situationAfterAction) ?situation .
            ?activity :action ?action ;
                      :action ?allActions .
            ?allActions ho:object ?object .
        }
    """
    sparql = SPARQLWrapper(auth[0])
    sparql.setHTTPAuth(BASIC)
    sparql.setCredentials(auth[1], auth[2])
    sparql.setQuery(queryString % state)
    sparql.setReturnFormat(JSON)

    try :
        json = sparql.query().convert()
        if json['boolean'] is True:
            result = False
        else:
            result = True
    except  Exception as e:
        print(e)
        result = e
    return result

In [7]:
def getTTLFileName(activity, scene):
    ttl_file_activity = activity.replace("http://example.org/virtualhome2kg/instance/","")
    ttl_file_activity = ttl_file_activity.replace("_"+scene, "")
    m = re.search(r'\d+',ttl_file_activity)
    activity_name = re.split(r'\d+',ttl_file_activity)[0]
    activity_id = m.group()
    file_name = "virtualhome2kg-" + activity_name.capitalize() + "-" + activity_id + ".ttl"
    return file_name

In [8]:
def replicateActivity(g, scene, ep_id):
    queryString = """
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        SELECT DISTINCT ?s WHERE {
            ?s a ?class .
            filter ((?class = ho:Activity) || (?class = :Event) || (?class = :StartEvent) || (?class =:EndEvent) || (?class = :Situation) || (?class = :State) || (?class = ho:Shape))
        }
    """
    qres = g.query(queryString)
    for res in qres:
        replica = res.s + "_ep" + str(ep_id)
        g.add((URIRef(replica),res.p,res.o))
    print(g.serialize('test.ttl',format="ttl"))
    #主語になるもののURIをすべて改変して複製

In [9]:
def update(g, activity1, activity2, scene, auth):
    result = None

    # activity2のttlを取得する
    file_name2 = getTTLFileName(activity2, scene)

    # ttlをロードして結合処理
    g.parse("rdf/" + scene + "/" + file_name2, format='turtle')
    queryString = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        DELETE {
            ?startState :partOf ?situation ;
                rdf:type :State ;
                :isStateOf ?object ;
                :nextState ?nextState .
        } INSERT {
            ?activity1 :nextActivity ?activity2 .
            ?endState :partOf ?situation ; 
                        :nextState ?nextState .
        } WHERE {
            BIND(<%activity1> AS ?activity1)
            ?activity1 :hasEvent ?endEvent .
            ?endEvent a :EndEvent .
            ?endEvent :situationAfterEvent ?endSituation .
            ?endState :partOf ?endSituation ;
                        :isStateOf ?object .
            BIND(<%activity2> AS ?activity2)
            ?activity2 :hasEvent ?startEvent .
            ?startEvent a :StartEvent .
            ?startEvent :situationBeforeEvent ?startSituation .
            ?startState :partOf ?startSituation ;
                        :isStateOf ?object .
            ?startState :partOf ?situation .
            optional {?startState :nextState ?nextState}
        }
    """
    queryString = queryString.replace("%activity1", activity1)
    queryString = queryString.replace("%activity2", activity2)

#     sparql = SPARQLWrapper(auth[0] + "/statements")
#     sparql.setHTTPAuth(BASIC)
#     sparql.setCredentials(auth[1], auth[2])
#     sparql.method = "POST"
#     sparql.queryType = "INSERT"
#     sparql.setQuery(queryString)
#     sparql.setReturnFormat(JSON)
#     try :
#         qres = g.query(queryString)
#         result = qres
    g.update(queryString)
#         json = sparql.query().convert()
#         bindings = json
#         result = bindings
#     except  Exception as e:
#         print(e)
    return g

In [10]:
def checkNextActivityIsRoutined(g, activity2):
    result = None
    queryString = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        ASK {
            ?prevActivity :nextActivity <%activity2> .
        }
    """
    
    try :
        qres = g.query(queryString.replace("%activity2", activity2))
        result = [row for row in qres][0]
    except  Exception as e:
        print(e)
    return result

In [11]:
def addNextState(g, activity1, activity2):
    result = None
    queryString = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        INSERT {
            ?prevState :nextState ?startState .
        } WHERE {
            <%activity1> :hasEvent ?endEvent .
            ?endEvent a :EndEvent ;
                    :situationAfterEvent ?endSituation .
            ?endState :partOf ?endSituation ;
                        :isStateOf ?object .
            ?prevState :nextState ?endState .
            BIND(<%activity2> AS ?activity2)
            ?activity2 :hasEvent ?startEvent .
            ?startEvent a :StartEvent ;
                        :situationBeforeEvent ?startSituation .
            ?startState :partOf ?startSituation ;
                        :isStateOf ?object .
        }
    """
    try :
        queryString = queryString.replace("%activity1", activity1)
        queryString = queryString.replace("%activity2", activity2)
        g.update(queryString)
    except  Exception as e:
        print(e.args)
    return g

In [12]:
def deleteFinalState(g, activity1, activity2):
    result = None
    queryString = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        DELETE {
            ?finalState ?p ?o .
        } WHERE {
            <%activity1> :hasEvent ?endEvent .
            ?endEvent a :EndEvent ;
                    :situationAfterEvent ?endSituation .
            ?endState :partOf ?endSituation ;
                        :isStateOf ?object .
            ?endState ?p ?o .
            <%activity2> :hasEvent ?startEvent .
            ?startEvent a :StartEvent ;
                        :situationBeforeEvent ?startSituation .
            ?startState :partOf ?startSituation ;
                        :isStateOf ?object .
        }
    """
    try :
        queryString = queryString.replace("%activity1", activity1)
        queryString = queryString.replace("%activity2", activity2)
        g.update(queryString)
    except  Exception as e:
        print(e.args)
    return g

In [13]:
def updateRoutine(g, activity1, activity2):
    result = None
    queryString = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        INSERT { 
            <%activity1> :nextActivity <%activity2> .
            ?startState :partOf ?situation .
        } WHERE {
            {
                <%activity1> :hasEvent ?endEvent .
                ?endEvent a :EndEvent ;
                            :situationAfterEvent ?endSituation .
                ?endState :partOf ?endSituation ;
                            :isStateOf ?object .

                <%activity2> :hasEvent ?startEvent .
                ?startEvent a :StartEvent ;
                            :situationBeforeEvent ?startSituation .
                ?startState :partOf ?startSituation ;
                            :isStateOf ?object .
            } union {
                <%activity1> :hasEvent ?endEvent .
                ?endEvent a :EndEvent ;
                            :situationAfterEvent ?endSituation .
                ?endState :partOf ?endSituation ;
                            :partOf ?situation .
            }
        }
    """
    try :
        queryString = queryString.replace("%activity1", activity1)
        queryString = queryString.replace("%activity2", activity2)
        g.update(queryString)
    except  Exception as e:
        print(e.args)
    return g

In [14]:
def preprocess(auth, activity, scene, activityMap, stateMap):
    #get objects' first/final states
    bindings = getFirstAndFinalStates(auth, activity, scene)

    try:
        #preprocess
        for elem in bindings:
            firstState = elem['startState']['value']
            finalState = elem['endState']['value']
            #firstStateに関してstateMapの作成
            if 'firstStateType' in elem:
                if firstState not in stateMap:
                    stateMap[firstState] = {
                        "state": [elem['startStateType']['value']],
                        "object": elem['endObject']['value']
                    }
                else:
                    if elem['startStateType']['value'] not in stateMap[firstState]['state']:
                        stateMap[firstState]['state'].append(elem['startStateType']['value'])
            else:
                if firstState not in stateMap:
                    stateMap[firstState] = {
                        "state": [],
                        "object": elem['startObject']['value']
                    }
                else:
                    pass

            #endStateに関してstateMapの作成
            if  'endStateType' in elem:
                if finalState not in stateMap:
                    stateMap[finalState] = {
                        "state": [elem['endStateType']['value']],
                        "object": elem['endObject']['value']
                    }
                else:
                    if elem['endStateType']['value'] not in stateMap[finalState]['state']:
                        stateMap[finalState]['state'].append(elem['endStateType']['value'])
            else:
                if finalState not in stateMap:
                    stateMap[finalState] = {
                        "state": [],
                        "object": elem['endObject']['value']
                    }
                else:
                    pass

            if activity in activityMap:
                if firstState not in activityMap[activity]['startState']:
                    activityMap[activity]['startState'].append(firstState)
                if finalState not in activityMap[activity]['endState']:
                    activityMap[activity]['endState'].append(finalState)
            else:
                #activity初出時
                activityMap[activity] = {
                    'startState': [firstState],
                    'endState': [finalState]
                }
    except Except as e:
        print(e.args)
    return activityMap, stateMap

In [15]:
def loadAuthFile():
    with open('auth.txt') as f:
        auth =  f.readlines()
        auth = [x.replace('\n','') for x in auth]
    return auth

In [16]:
inverse_states = {
    "http://example.org/virtualhome2kg/ontology/CLOSED": "http://example.org/virtualhome2kg/ontology/OPEN",
    "http://example.org/virtualhome2kg/ontology/OFF": "http://example.org/virtualhome2kg/ontology/ON",
    "http://example.org/virtualhome2kg/ontology/DIRTY": "http://example.org/virtualhome2kg/ontology/CLEAN",
    "http://example.org/virtualhome2kg/ontology/PLUGGED_IN": "http://example.org/virtualhome2kg/ontology/PLUGGED_OUT",
    "http://example.org/virtualhome2kg/ontology/OPEN": "http://example.org/virtualhome2kg/ontology/CLOSED",
    "http://example.org/virtualhome2kg/ontology/ON": "http://example.org/virtualhome2kg/ontology/OFF",
    "http://example.org/virtualhome2kg/ontology/CLEAN": "http://example.org/virtualhome2kg/ontology/DIRTY",
    "http://example.org/virtualhome2kg/ontology/PLUGGED_OUT": "http://example.org/virtualhome2kg/ontology/PLUGGED_IN"
}

In [17]:
auth = loadAuthFile()
episode_list = readRoutine('graph_state_list/episode_list1_scene4_20210826.txt')
scene = "scene4"
executable_routine = 0
executable_episode_list = []
executable_episode_desc_list = []
base = Namespace("http://example.org/virtualhome2kg/instance/")
onto = Namespace("http://example.org/virtualhome2kg/ontology/")
# replicateActivity("http://example.org/virtualhome2kg/instance/bring_dirty_plate_to_sink0_scene4", scene,1)
episode_num = 1
for episode in episode_list:
    activity_list = []
    activity_desc_list = []
    for activity in episode:
        activity = activity.replace(' ', '_')
        activity = activity.lower()
        activity_description = getActivity(auth, activity, scene)
        activity_list.append(activity_description[0])
        activity_desc_list.append(activity_description[1])
    activityMap = {}
    stateMap = {}

    #check consistency
    for i in range(len(activity_list)-1):
        activity1 = activity_list[i]
        activity2 = activity_list[i+1]
        activityMap, stateMap = preprocess(auth, activity1, scene, activityMap, stateMap)
        activityMap, stateMap = preprocess(auth, activity2, scene, activityMap, stateMap)
        consistency = True
        statesChanged = True
        activity1_value = None
        activity2_value = None
        if activity1 == activity2:
            consistency = False
            break
        
        if activity1 in activityMap:
            activity1_value = activityMap[activity1]
            if activity2 in activityMap:
                activity2_value = activityMap[activity2]
            else:
                # if there is no state changed
                statesChanged = False
        else:
            # if there is no state changed
            statesChanged = False

        #print([activity1, activity2])
        try:
            if statesChanged:
                assert checkConsistency(activity1_value, activity2_value)
        except Exception as e:
            consistency = False
            break
        
    #SPARQL DELETE/INSERT query
    if consistency:
        print("Executable")
        print(episode)
        executable_routine += 1
        executable_episode_list.append(episode)
        executable_episode_desc_list.append(activity_desc_list)
        g = Graph()
        for i in range(len(activity_list)-1):
            activity1 = activity_list[i]
            activity2 = activity_list[i+1]
            if checkNextActivityIsRoutined(g, activity2):
                g = deleteFinalState(g, activity1, activity2)
                g = updateRoutine(g, activity1, activity2)
                g = addNextState(g, activity1,activity2)
                print(str(i) + " updatedRoutine: " + activity1 + " " + activity2)
            else:
                if i == 0:
                    file_name1 = getTTLFileName(activity1, scene)
                    g.parse("rdf/" + scene + "/" + file_name1, format='turtle')
                g = update(g, activity1=activity1, activity2=activity2, scene=scene, auth=auth)
            # qres = update(g, activity1=activity1, activity2=activity2, scene=scene)
            # for res in qres:
            #     print(res)
            print(str(i) + " updated: " + activity1 + " " + activity2)
        
        # create episode resource
        episode_r = base["episode" + str(episode_num)]
        g.add((episode_r, RDF.type, onto.Episode))
        activity_num = 1
        for activity in activity_list:
            g.add((episode_r, onto.hasActivity, URIRef(activity)))
            g.add((URIRef(activity), onto.activitiyNumber, Literal(activity_num, datatype=XSD.int)))
            activity_num += 1
        
        ttl_file = "episode" + str(episode_num) + "_" + scene + "_20211003.ttl"
        g.serialize(destination=ttl_file, format="turtle")
        
        # change URI
        data_lines = None
        with open(ttl_file) as f:
            data_lines = f.read()
        for activity in activity_list:
            activity_name = activity.replace("http://example.org/virtualhome2kg/instance/","")
            if activity_name + "_ep" not in data_lines:
                data_lines = data_lines.replace(activity_name, activity_name + "_ep" + str(episode_num))
        with open(ttl_file, mode="w") as f:
            f.write(data_lines)
        
        episode_num += 1
#         sys.exit()
    else:
        print("not executable")

Executable
['Eat dinner', 'Bring dirty plate to sink', 'Browse internet', 'Brush teeth', 'Change toilet paper roll', 'Brush teeth']
0 updated: http://example.org/virtualhome2kg/instance/eat_dinner0_scene4 http://example.org/virtualhome2kg/instance/bring_dirty_plate_to_sink0_scene4
1 updated: http://example.org/virtualhome2kg/instance/bring_dirty_plate_to_sink0_scene4 http://example.org/virtualhome2kg/instance/browse_internet23_scene4
2 updated: http://example.org/virtualhome2kg/instance/browse_internet23_scene4 http://example.org/virtualhome2kg/instance/brush_teeth1_scene4
3 updated: http://example.org/virtualhome2kg/instance/brush_teeth1_scene4 http://example.org/virtualhome2kg/instance/change_toilet_paper_roll0_scene4
4 updated: http://example.org/virtualhome2kg/instance/change_toilet_paper_roll0_scene4 http://example.org/virtualhome2kg/instance/brush_teeth0_scene4
Executable
['Make popcorn', 'Drink', 'Clean', 'Empty dishwasher and fill dishwasher', 'Cutting', 'Grab some juice']
0 up

Executable
['Cook some food', 'Eat dinner', 'Listen to music', 'Read book', 'Go to sleep', 'Browse internet']
0 updated: http://example.org/virtualhome2kg/instance/cook_some_food8_scene4 http://example.org/virtualhome2kg/instance/eat_dinner0_scene4
1 updated: http://example.org/virtualhome2kg/instance/eat_dinner0_scene4 http://example.org/virtualhome2kg/instance/listen_to_music16_scene4
2 updated: http://example.org/virtualhome2kg/instance/listen_to_music16_scene4 http://example.org/virtualhome2kg/instance/read_book29_scene4
3 updated: http://example.org/virtualhome2kg/instance/read_book29_scene4 http://example.org/virtualhome2kg/instance/go_to_sleep2_scene4
4 updated: http://example.org/virtualhome2kg/instance/go_to_sleep2_scene4 http://example.org/virtualhome2kg/instance/browse_internet23_scene4
Executable
['Listen to music', 'Browse internet', 'Relax on sofa', 'Browse internet', 'Added meat to freezer', 'Read book']
0 updated: http://example.org/virtualhome2kg/instance/listen_to_mus

In [18]:
activityMap

{'http://example.org/virtualhome2kg/instance/browse_internet20_scene4': {'startState': ['http://example.org/virtualhome2kg/instance/state0_home_office267_browse_internet20_scene4'],
  'endState': ['http://example.org/virtualhome2kg/instance/state14_computer292_browse_internet20_scene4']},
 'http://example.org/virtualhome2kg/instance/complete_surveys_on_amazon_turk0_scene4': {'startState': ['http://example.org/virtualhome2kg/instance/state0_chair68_complete_surveys_on_amazon_turk0_scene4'],
  'endState': ['http://example.org/virtualhome2kg/instance/state16_keyboard1001_complete_surveys_on_amazon_turk0_scene4']},
 'http://example.org/virtualhome2kg/instance/cook_some_food5_scene4': {'startState': ['http://example.org/virtualhome2kg/instance/state0_dining_room11_cook_some_food5_scene4'],
  'endState': ['http://example.org/virtualhome2kg/instance/state18_oven105_cook_some_food5_scene4',
   'http://example.org/virtualhome2kg/instance/state18_food_food1000_cook_some_food5_scene4']},
 'http:/

In [19]:
executable_routine

19